### This code taken and/or adapted from [QuantPY](https://quantpy.com.au/)

In [1]:
# import os
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr

# import scipy as sci
from scipy import stats
import scipy.optimize as sco

import yfinance as yf

import datetime as dt
from datetime import date
from datetime import timedelta

import plotly.graph_objects as go

In [2]:
# Specify date range for data
# Set start and end dates of 5 years "5Y" back from your current date
# Define the time delta (time-frame) to start with
time_delta = 10

end_date = pd.to_datetime('today')
# 10 years
start_date = end_date - np.timedelta64(time_delta, 'Y')
start, end = start_date, end_date
# Verify date range
print(f"Start date: {start} and End date: {end}")

Start date: 2012-10-24 06:50:07.400697 and End date: 2022-10-24 17:02:07.400697


In [3]:
# Specify portfolio sectors - basket of market index and assets
blend_list = ['IBM', 'TSLA','TSM', 'NVDA', 'DVN', 'OXY','CNI', 'NOC','BHF', 'AIG']
tech_list = ['AAPL', 'IBM', 'TSLA', 'GOOGL', 'MSFT','CSCO', 'TSM', 'NVDA']
energy_list = ['DVN', 'OXY', 'NEE', 'MRO', 'APA', 'TRGP', 'PSX', 'EQT']
industrials_list = ['BA', 'GE', 'CNI', 'NOC', 'LPX', 'BLDR', 'AAL', 'CPA', 'CAR', 'CPA', 'HXL']
financials_list = ['BHF', 'AIG', 'EQH', 'PFG', 'WBS', 'UNM', 'FHN', 'LPLA']
market = ['^GSPC']   # adding S&P 500 Index (^GSPC) for use in beta weighting

In [4]:
# Specify portfolio sectors - basket of market index and assets - S&P 500 Index (^GSPC) and Tech Stocks List
basket_list = market + tech_list + energy_list + industrials_list + financials_list 

print("Basket of stocks")
print(basket_list)

Basket of stocks
['^GSPC', 'AAPL', 'IBM', 'TSLA', 'GOOGL', 'MSFT', 'CSCO', 'TSM', 'NVDA', 'DVN', 'OXY', 'NEE', 'MRO', 'APA', 'TRGP', 'PSX', 'EQT', 'BA', 'GE', 'CNI', 'NOC', 'LPX', 'BLDR', 'AAL', 'CPA', 'CAR', 'CPA', 'HXL', 'BHF', 'AIG', 'EQH', 'PFG', 'WBS', 'UNM', 'FHN', 'LPLA']


#### Get Stock Data

In [5]:
# Import data
def getData(basket_list, start_date, end_date):
    stockData = pdr.get_data_yahoo(basket_list, start = start_date, end = end_date)
    stockData = stockData['Close']

    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

# call it
meanReturns, covMatrix = getData(basket_list, start_date, end_date)
display(meanReturns)
display(covMatrix)

Symbols
^GSPC    0.000455
AAPL     0.000928
IBM     -0.000016
TSLA     0.002521
GOOGL    0.000854
MSFT     0.001009
CSCO     0.000493
TSM      0.000725
NVDA     0.001857
DVN      0.000613
OXY      0.000437
NEE      0.000669
MRO      0.000549
APA      0.000410
TRGP     0.000717
PSX      0.000557
EQT      0.000517
BA       0.000562
GE      -0.000097
CNI      0.000481
NOC      0.000916
LPX      0.000819
BLDR     0.001493
AAL      0.000547
CPA      0.000361
CAR      0.001954
HXL      0.000575
BHF      0.000326
AIG      0.000381
EQH      0.000676
PFG      0.000616
WBS      0.000603
UNM      0.000573
FHN      0.000617
LPLA     0.001149
dtype: float64

Symbols,^GSPC,AAPL,IBM,TSLA,GOOGL,MSFT,CSCO,TSM,NVDA,DVN,...,CAR,HXL,BHF,AIG,EQH,PFG,WBS,UNM,FHN,LPLA
Symbols,,,,,,,,,,,,,,,,,,,,,
^GSPC,0.000120,0.000140,0.000104,0.000169,0.000134,0.000142,0.000121,0.000121,0.000191,0.000180,...,0.000221,0.000144,0.000301,0.000148,0.000276,0.000162,0.000161,0.000166,0.000141,0.000155
AAPL,0.000140,0.000333,0.000105,0.000236,0.000170,0.000187,0.000144,0.000166,0.000262,0.000170,...,0.000207,0.000127,0.000281,0.000132,0.000268,0.000148,0.000144,0.000146,0.000119,0.000154
IBM,0.000104,0.000105,0.000219,0.000110,0.000103,0.000113,0.000124,0.000100,0.000136,0.000177,...,0.000200,0.000141,0.000316,0.000147,0.000275,0.000159,0.000162,0.000181,0.000141,0.000135
TSLA,0.000169,0.000236,0.000110,0.001270,0.000215,0.000224,0.000149,0.000222,0.000371,0.000259,...,0.000384,0.000201,0.000358,0.000180,0.000340,0.000178,0.000184,0.000175,0.000182,0.000236
GOOGL,0.000134,0.000170,0.000103,0.000215,0.000278,0.000191,0.000130,0.000150,0.000248,0.000161,...,0.000223,0.000137,0.000279,0.000137,0.000254,0.000148,0.000142,0.000140,0.000117,0.000159
MSFT,0.000142,0.000187,0.000113,0.000224,0.000191,0.000282,0.000152,0.000154,0.000270,0.000160,...,0.000218,0.000130,0.000273,0.000132,0.000253,0.000150,0.000141,0.000143,0.000115,0.000156
CSCO,0.000121,0.000144,0.000124,0.000149,0.000130,0.000152,0.000255,0.000122,0.000198,0.000174,...,0.000211,0.000132,0.000289,0.000136,0.000256,0.000158,0.000156,0.000164,0.000133,0.000146
TSM,0.000121,0.000166,0.000100,0.000222,0.000150,0.000154,0.000122,0.000338,0.000274,0.000182,...,0.000207,0.000140,0.000291,0.000126,0.000270,0.000144,0.000147,0.000155,0.000121,0.000155
NVDA,0.000191,0.000262,0.000136,0.000371,0.000248,0.000270,0.000198,0.000274,0.000756,0.000229,...,0.000323,0.000188,0.000405,0.000173,0.000373,0.000196,0.000196,0.000196,0.000158,0.000222


In [6]:
df = pdr.get_data_yahoo(basket_list, start_date, end_date)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

Symbols,^GSPC,AAPL,IBM,TSLA,GOOGL,MSFT,CSCO,TSM,NVDA,DVN,...,CAR,HXL,BHF,AIG,EQH,PFG,WBS,UNM,FHN,LPLA
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-11,0.001706,-0.007659,-0.000693,-0.013068,-0.001892,-0.002147,-0.008023,0.003514,-0.021763,0.001466,...,0.007865,0.004317,-0.011573,0.001894,0.050334,0.004755,0.003458,-0.001580,0.006851,-0.001292
2018-05-14,0.000883,-0.002336,0.001109,-0.030659,0.002914,0.003372,-0.005020,0.001002,0.003256,0.008995,...,0.000000,0.004442,0.014240,0.001324,0.005129,0.001354,-0.002671,-0.001846,-0.007380,0.001436
2018-05-15,-0.006866,-0.009130,-0.003888,-0.027043,-0.019832,-0.007269,-0.004826,-0.016404,-0.039133,-0.000969,...,-0.008330,-0.001860,-0.007817,0.005841,0.004640,0.000846,0.009084,0.006053,0.015748,0.000717
2018-05-16,0.004052,0.009289,0.006173,0.008061,-0.000719,-0.001748,-0.007061,0.012391,0.001831,0.002903,...,0.017960,0.001002,-0.017500,0.008977,0.000000,0.004554,0.006062,0.009402,0.013451,-0.000861
2018-05-17,-0.000856,-0.006344,-0.000899,-0.006795,-0.002614,-0.010035,-0.038371,-0.021080,0.006887,0.022924,...,0.021001,0.007697,0.008163,0.019909,0.003697,0.002689,0.005255,0.010087,0.004103,0.005438


### Directly calculate beta

In [7]:
def calc_beta(df):
    np_array = df.values
    # Market index is the first column 0
    m = np_array[:,0] 
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            # stock returns are indexed by ind
            s = np_array[:,ind] 
            # Calculate covariance matrix between stock and market
            covariance = np.cov(s,m) 
            beta.append( covariance[0,1]/covariance[1,1] )
    return pd.Series(beta, df.columns[1:], name='Beta')

beta = calc_beta(log_returns)

### Define your Portfolio and make DataFrame
Calculate Beta Weighted Portfolio

In [8]:
units = np.array([100, 250, 300, 400, 200])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = [unit*pr for unit, pr in zip(units, price)]
weight = [round(val/sum(value),2) for val in value]
beta = round(beta,2)

Portfolio = pd.DataFrame({
    'Stock': basket_list,
    'Direction': 'Long',
    'Type': 'S',
    'Stock Price': price,
    'Price': price,
    'Units': units,
    'Value': units*price,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

ValueError: operands could not be broadcast together with shapes (5,) (34,) 

### Weight the Delta’s using Beta

In [ ]:
Portfolio['ASX200 Weighted Delta (point)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']/ASXprices[0]) * Portfolio['Delta'],2) 
Portfolio['ASX200 Weighted Delta (1%)'] = round(Portfolio['Beta'] * (Portfolio['Stock Price']) * Portfolio['Delta'] * 0.01,2) 
Portfolio

### Total the Delta’s to get Portfolio Overview

In [ ]:
Portfolio.loc['Total', ['Value', 'ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']] \
= Portfolio[['Value','ASX200 Weighted Delta (point)', 'ASX200 Weighted Delta (1%)']].sum()
Portfolio

### calculate portfolioPerformance metrics. 

In [ ]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(weights.T,np.dot(covMatrix, weights))
           )*np.sqrt(252)
    return returns, std

# call it
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

### Maximium Sharpe Ratio Portfolio

In [ ]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

### Minimium Portfolio Variance

In [ ]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

### Creating the Efficient Frontier

In [ ]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

### callable function for graphing package like Plotly/Dash combination

In [ ]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0.03, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList

### Visualising the Efficient Frontier

In [ ]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()